## Download Libraries

In [ ]:
!pip install pyLDAvis

In [ ]:
!pip install h2o

In [ ]:
!pip install gensim

## Import libraries

In [1]:
import pandas as pd
import re
import numpy as np
import string
from collections import Counter
from pprint import pprint

#Gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
from gensim.models import CoherenceModel

import matplotlib.pyplot as plt

# NLTK
import nltk
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# H2O
import h2o
from h2o.estimators import H2ORandomForestEstimator
h2o.init()

# Sci-kit Learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

# Plotting Libraries
import pyLDAvis
import pyLDAvis.gensim_models

Checking whether there is an H2O instance running at http://localhost:54321 .

c:\miniconda37\lib\site-packages\h2o\backend\connection.py:371: UserWarning: Proxy is defined in the environment: HTTP_PROXY. This may interfere with your H2O Connection.
  "This may interfere with your H2O Connection." % name)


 connected.


H2O_cluster_uptime:,2 days 3 hours 33 mins
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.2
H2O_cluster_version_age:,"28 days, 22 hours and 2 minutes"
H2O_cluster_name:,H2O_from_python_A174411_lufbsv
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,87.5 Mb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [2]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\A174411\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\A174411\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\A174411\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\A174411\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## Load Dataset and Make Copy of File

In [3]:
remedy_file0 = pd.read_csv(r"C:\Users\A174411\Onedrive - Progressive Corporation\groupProject\UPDATEDCustom SQL Query.csv", dtype='object')

In [4]:
remedy_file0.head()

,Assignee,Notes_Text,Priority,Record Type,RecordID,Reported_Date_Time,Resolution_Text,Resolved_Date_Time,Status,Status Reason,Submission Type,Summary_Text,Support_Group,Support_Org
0,NaN,SRQDDP05 Reported no UF data for past hour. Se...,High,Incident,INC000011182959,2/1/2021 10:09:29 AM,NaN,2/1/2021 4:49:34 PM,Closed,Automated Resolution Reported,Automated,SRQDDP05 Reported no UF data for past hour,Systems & Infrastructure T2,Computing
1,NaN,A web test monitor returned an error code that...,High,Incident,INC000011644117,7/9/2021 12:38:21 AM,NaN,7/9/2021 12:43:17 AM,Closed,Automated Resolution Reported,Automated,SRQDDP05 SCOMURL Reported error Content Matche...,Systems & Infrastructure T2,Computing
2,NaN,A web test monitor returned an error code that...,High,Incident,INC000011662455,7/16/2021 12:38:23 AM,NaN,7/16/2021 12:41:14 AM,Closed,Automated Resolution Reported,Automated,SRQDDP05 SCOMURL Reported error Content Matche...,Systems & Infrastructure T2,Computing
3,NaN,SRQDDP06 Reported no UF data for past hour. Se...,High,Incident,INC000011182820,2/1/2021 10:09:19 AM,NaN,2/1/2021 3:23:47 PM,Closed,Automated Resolution Reported,Automated,SRQDDP06 Reported no UF data for past hour,Systems & Infrastructure T2,Computing
4,NaN,SRQDDP07 Reported no UF data for past hour. Se...,High,Incident,INC000011182756,2/1/2021 10:09:21 AM,NaN,2/1/2021 4:03:26 PM,Closed,Automated Resolution Reported,Automated,SRQDDP07 Reported no UF data for past hour,Systems & Infrastructure T2,Computing


In [5]:
remedy_file0['Notes_Text'].head()

0    SRQDDP05 Reported no UF data for past hour. Se...
1    A web test monitor returned an error code that...
2    A web test monitor returned an error code that...
3    SRQDDP06 Reported no UF data for past hour. Se...
4    SRQDDP07 Reported no UF data for past hour. Se...
Name: Notes_Text, dtype: object

In [6]:
remedy_file0['Summary_Text'].head()

0           SRQDDP05 Reported no UF data for past hour
1    SRQDDP05 SCOMURL Reported error Content Matche...
2    SRQDDP05 SCOMURL Reported error Content Matche...
3           SRQDDP06 Reported no UF data for past hour
4           SRQDDP07 Reported no UF data for past hour
Name: Summary_Text, dtype: object

In [7]:
remedy_file = remedy_file0.copy(deep=True)
print(remedy_file.shape)
print(remedy_file.columns)

(211149, 14)
Index(['Assignee', 'Notes_Text', 'Priority', 'Record Type', 'RecordID',
       'Reported_Date_Time', 'Resolution_Text', 'Resolved_Date_Time', 'Status',
       'Status Reason', 'Submission Type', 'Summary_Text', 'Support_Group',
       'Support_Org'],
      dtype='object')


In [8]:
remedy_file['Resolution_Text'].tail()

211144    Customer confirmed server environment normaliz...
211145    Transfer appears successful, ticket can be re-...
211146    SI - Contacted Customer, was able to resolve i...
211147          Resolved after multiple-day IMTs, see notes
211148                              Kyle resolved the issue
Name: Resolution_Text, dtype: object

## Clean, Stem, and Lemmatize the data for LDA

<b> Summary Text

In [9]:
stemmer = SnowballStemmer("english")

In [10]:
def lemmatize_stemming(comment):
    return stemmer.stem(WordNetLemmatizer().lemmatize(comment, pos='v'))
def preprocess(comment):
    result = []
    for token in gensim.utils.simple_preprocess(comment):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [11]:
remedy_file["clean_Summary"] = remedy_file['Summary_Text']

In [12]:
# REPLACES NULL WITH NaN
remedy_file["clean_Summary"].replace('', np.nan, inplace=True)
#remedy_file.head()

# DROPS NaN
remedy_file["clean_Summary"].dropna()
remedy_file["clean_Summary"].head()

0           SRQDDP05 Reported no UF data for past hour
1    SRQDDP05 SCOMURL Reported error Content Matche...
2    SRQDDP05 SCOMURL Reported error Content Matche...
3           SRQDDP06 Reported no UF data for past hour
4           SRQDDP07 Reported no UF data for past hour
Name: clean_Summary, dtype: object

In [13]:
processed_Summary = remedy_file["clean_Summary"].map(preprocess) # map returns a list of the results after applying the given function to each item of a given iterable (list, tuple etc.)

In [14]:
processed_Summary[:10]

0                   [srqddp, report, data, past, hour]
1    [srqddp, scomurl, report, error, content, matc...
2    [srqddp, scomurl, report, error, content, matc...
3                   [srqddp, report, data, past, hour]
4                   [srqddp, report, data, past, hour]
5    [srqddp, scomurl, report, error, content, matc...
6    [srqddp, scomurl, report, error, content, matc...
7                   [srqddp, report, data, past, hour]
8                   [srqddp, report, data, past, hour]
9                   [srqdwg, report, data, past, hour]
Name: clean_Summary, dtype: object

<b> Resolution Text

In [15]:
remedy_file["clean_Resolution"] = remedy_file['Resolution_Text'].str.replace('[^\w\s]','')

<>:1: DeprecationWarning: invalid escape sequence \w
<>:1: DeprecationWarning: invalid escape sequence \w
<>:1: DeprecationWarning: invalid escape sequence \w
C:\Users\A174411\AppData\Local\Temp\ipykernel_23004\3330310722.py:1: DeprecationWarning: invalid escape sequence \w
  remedy_file["clean_Resolution"] = remedy_file['Resolution_Text'].str.replace('[^\w\s]','')
c:\miniconda37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [16]:
# REPLACES NULL WITH NaN
remedy_file["clean_Resolution"].replace('', np.NaN, inplace=True)
#remedy_file.head()

# DROPS NaN
remedy_file["clean_Resolution"].dropna()
remedy_file["clean_Resolution"].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: clean_Resolution, dtype: object

In [17]:
remedy_file["clean_Resolution"] =str(remedy_file["clean_Resolution"])

In [18]:
processed_Resolution = remedy_file["clean_Resolution"].map(preprocess)

In [19]:
processed_Resolution[:10]

0    [custom, confirm, server, environ, normaliz, t...
1    [custom, confirm, server, environ, normaliz, t...
2    [custom, confirm, server, environ, normaliz, t...
3    [custom, confirm, server, environ, normaliz, t...
4    [custom, confirm, server, environ, normaliz, t...
5    [custom, confirm, server, environ, normaliz, t...
6    [custom, confirm, server, environ, normaliz, t...
7    [custom, confirm, server, environ, normaliz, t...
8    [custom, confirm, server, environ, normaliz, t...
9    [custom, confirm, server, environ, normaliz, t...
Name: clean_Resolution, dtype: object

<b> Notes Text

In [20]:
remedy_file["clean_Notes"] = remedy_file['Notes_Text'].str.replace('[^\w\s]','')

<>:1: DeprecationWarning: invalid escape sequence \w
<>:1: DeprecationWarning: invalid escape sequence \w
<>:1: DeprecationWarning: invalid escape sequence \w
C:\Users\A174411\AppData\Local\Temp\ipykernel_23004\2073217072.py:1: DeprecationWarning: invalid escape sequence \w
  remedy_file["clean_Notes"] = remedy_file['Notes_Text'].str.replace('[^\w\s]','')
c:\miniconda37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [21]:
remedy_file["clean_Notes"] =str(remedy_file["clean_Notes"])

In [22]:
# REPLACES NULL WITH NaN
remedy_file["clean_Notes"].replace('', np.NaN, inplace=True)
#remedy_file.head()

# DROPS NaN
remedy_file["clean_Notes"].dropna()
remedy_file["clean_Notes"].head()

0    0         SRQDDP05 Reported no UF data for pas...
1    0         SRQDDP05 Reported no UF data for pas...
2    0         SRQDDP05 Reported no UF data for pas...
3    0         SRQDDP05 Reported no UF data for pas...
4    0         SRQDDP05 Reported no UF data for pas...
Name: clean_Notes, dtype: object

In [23]:
processed_Notes = remedy_file["clean_Notes"].map(preprocess)

In [24]:
processed_Notes[:10]

0    [srqddp, report, data, past, hour, test, monit...
1    [srqddp, report, data, past, hour, test, monit...
2    [srqddp, report, data, past, hour, test, monit...
3    [srqddp, report, data, past, hour, test, monit...
4    [srqddp, report, data, past, hour, test, monit...
5    [srqddp, report, data, past, hour, test, monit...
6    [srqddp, report, data, past, hour, test, monit...
7    [srqddp, report, data, past, hour, test, monit...
8    [srqddp, report, data, past, hour, test, monit...
9    [srqddp, report, data, past, hour, test, monit...
Name: clean_Notes, dtype: object

## Clean and Lemmatize the Data for TF-IDFVectorizer

<b> Summary Text

In [ ]:
# # CLEANS THE TEXT
# # clean_remedy_file = remedy_file.translate(str.maketrans('', '', string.punctuation)) #Error. 'DataFrame' object has no attribute 'translate'

# remedy_file["clean_Summary"] = remedy_file['Summary_Text'].str.replace('[^\w\s]',' ') #works!
# remedy_file["clean_Notes"] = remedy_file['Notes_Text'].str.replace('[^\w\s]','')
# remedy_file["clean_Resolution"] = remedy_file['Resolution_Text'].str.replace('[^\w\s]','') #works!
# remedy_file["clean_Resolution"].tail()

In [ ]:
# # Need word lemmatizing / stemming
# lemmatizer = WordNetLemmatizer()
# remedy_file["clean_Summary"] = [lemmatizer.lemmatize(comment) for comment in remedy_file["clean_Summary"]]

In [39]:
# # BREAK ROW STRING INTO INDIVIDUAL WORDS
remedy_file['tokenized_Summary'] = remedy_file["clean_Summary"].apply(word_tokenize)
# # remedy_file['tokenized_Notes'] = remedy_file["clean_Notes"].apply(word_tokenize)

In [40]:
# REMOVE STOPWORDS FROM LISTS AND APPEND RESULT TO FILTERED_TEXT
stop_words = set(stopwords.words('english'))
Summary_tokens = remedy_file['tokenized_Summary']
# test_list = word_tokens[:10]
Summary_filtered_comments = []

counter = 0

# for each comment (list)
for comment in Summary_tokens:
#for comment in work_tokens:
    counter += 1
    # print('Comment counter', counter)
    # print('comment', comment)
    # make an empty list to hold the nonstopwords of the specific comment 
    Summary_filtered_words = []
    # for each individual word in the comment list
    for word in comment:
    # if the individual word in the current comment is NOT a stop word, add it the filtered_words list
        # print("word:", word)
        if str(word).lower() not in stop_words:
            if str(word).lower() == 'reported':
                continue
            else:
                Summary_filtered_words.append(word.lower())
    # print('finished parsing commment, final ftilered words list of this specifc comment:', filtered_words,)
    Summary_filtered_comments.append(Summary_filtered_words)
    # print("main comment list", filtered_comments)
    # print()
#filtered_Text

In [41]:
flat_Summary = []

for lists in Summary_filtered_comments:
    for words in lists:
        flat_Summary.append(words)
#flat_list

## Frequency Distribution Plot
<b>Summary Text

In [ ]:
fdist_Summary = FreqDist(flat_Summary)
fdist_Summary

In [ ]:
fdist_Summary.most_common(2)

In [ ]:
fdist_Summary.plot(30 , cumulative=False)
# fdist_Summary()
# consider using a bar graph instead

In [ ]:
# # Plot frequency
# counts = Counter(flat_Summary)

# labels, values = zip(*Counter(flat_Summary).items())

# indexes = np.arange(len(labels))[:30]
# width = 1

# plt.bar(indexes, values, width)
# plt.xticks(indexes + width * 0.5, labels)
# plt.show()

## Counts of most common line in Summary Text and Resolution Text

In [ ]:
counts_Summary = Counter(remedy_file["clean_Summary"])

In [ ]:
counts_Summary.most_common(30)

In [ ]:
counts_Resolution = Counter(remedy_file["clean_Resolution"])

In [ ]:
counts_Resolution.most_common(30)

In [ ]:
# sentiment analysis(?)

## TF-IDF Vectorizer 
<b>Term Frequency–Inverse Document Frequency
<br>https://monkeylearn.com/blog/what-is-tf-idf/

In [42]:
X, y, x_test, y_test = train_test_split(
    remedy_file["clean_Summary"], remedy_file0,
    test_size=0.3,
    random_state=50
)

In [43]:
tfidf_vectorizer_l2 = TfidfVectorizer(ngram_range=(1,3), max_df=0.7, max_features= 300, norm="l2") 

In [44]:
X_tfidf_l2 = tfidf_vectorizer_l2.fit_transform(X)

In [45]:
tfidf_df_l2 = pd.DataFrame(
    [tfidf_vectorizer_l2.vocabulary_.keys(), tfidf_vectorizer_l2.idf_]
).T
tfidf_df_l2.columns = ["word", "tfidf_l2"]
tfidf_df_l2.sort_values(by=["tfidf_l2"], ascending=False, inplace=True)

In [46]:
print(type(tfidf_vectorizer_l2))
print(type(X_tfidf_l2))
print(tfidf_df_l2.head(30))

<class 'sklearn.feature_extraction.text.TfidfVectorizer'>
<class 'scipy.sparse.csr.csr_matrix'>
                      word  tfidf_l2
188          user prog1 on  5.981991
138           cn2 reported  5.660136
176             down value  5.549287
89               netbackup  5.549287
90                     job  5.549287
217            on bwillb1a  5.549287
175            forced down  5.549287
262  acvcsat1 reported pgr  5.545455
254        interconnect is  5.545455
74    reported error error  5.545455
267                  audit  5.544818
13                 is down  5.539102
178                by user  5.534048
177               value by  5.534048
88              logwatcher  5.533418
181         prog1 com node  5.529647
33                 uf data   5.52902
157                  lsass  5.523393
273           rate backup3  5.516559
134         down on server  5.511002
266                     cv  5.504251
206          of node value  5.500588
202             disable of  5.495725
39             u

## LDA using TF-IDF

<b> Summary Text

In [25]:
dict_Sum = corpora.Dictionary(processed_Summary)
count = 0
for k, v in dict_Sum.iteritems(): # k -> key | v -> value
    print(k, v)
    count += 1
    if count > 10:
        break

0 data
1 hour
2 past
3 report
4 srqddp
5 content
6 error
7 load
8 match
9 monitor
10 scomurl


In [26]:
dict_Sum.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
Sum_bow_corpus = [dict_Sum.doc2bow(doc) for doc in processed_Summary]

In [27]:
Sum_tfidf = models.TfidfModel(Sum_bow_corpus)
Sum_corpus_tfidf = Sum_tfidf[Sum_bow_corpus]

for comment in Sum_corpus_tfidf:
    pprint(comment)
    break

[(0, 0.28658918363462815),
 (1, 0.2899181622603864),
 (2, 0.2899116570409312),
 (3, 0.8658899064702043)]


In [49]:
Sum_lda_model_tfidf = gensim.models.LdaMulticore(Sum_corpus_tfidf, num_topics=4, id2word=dict_Sum, passes=2, workers=4)
for idx, topic in Sum_lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.069*"data" + 0.069*"hour" + 0.069*"past" + 0.062*"interfac" + 0.030*"mdsbesea" + 0.026*"crcdelta" + 0.026*"ethernierrstat" + 0.026*"slot" + 0.024*"chassi" + 0.018*"mdsh"
Topic: 1 Word: 0.098*"logwatch" + 0.061*"critic" + 0.058*"netbackup" + 0.048*"error" + 0.042*"ping" + 0.025*"vcenter" + 0.024*"vcvdip" + 0.024*"proghsz" + 0.021*"path" + 0.021*"datastor"
Topic: 2 Word: 0.031*"vcenter" + 0.030*"path" + 0.030*"datastor" + 0.027*"server" + 0.025*"proghsz" + 0.023*"prci" + 0.023*"prog" + 0.023*"acvcsap" + 0.021*"servic" + 0.021*"user"
Topic: 3 Word: 0.051*"servic" + 0.038*"critic" + 0.036*"sccl" + 0.033*"srcl" + 0.024*"ciscouc" + 0.024*"alert" + 0.023*"port" + 0.023*"fabric" + 0.020*"process" + 0.017*"logicalvolum"


In [50]:
for index, score in sorted(Sum_lda_model_tfidf[Sum_bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, Sum_lda_model_tfidf.print_topic(index, 10)))


Score: 0.7307319641113281	 
Topic: 0.051*"servic" + 0.038*"critic" + 0.036*"sccl" + 0.033*"srcl" + 0.024*"ciscouc" + 0.024*"alert" + 0.023*"port" + 0.023*"fabric" + 0.020*"process" + 0.017*"logicalvolum"

Score: 0.1855546087026596	 
Topic: 0.098*"logwatch" + 0.061*"critic" + 0.058*"netbackup" + 0.048*"error" + 0.042*"ping" + 0.025*"vcenter" + 0.024*"vcvdip" + 0.024*"proghsz" + 0.021*"path" + 0.021*"datastor"

Score: 0.041954413056373596	 
Topic: 0.069*"data" + 0.069*"hour" + 0.069*"past" + 0.062*"interfac" + 0.030*"mdsbesea" + 0.026*"crcdelta" + 0.026*"ethernierrstat" + 0.026*"slot" + 0.024*"chassi" + 0.018*"mdsh"

Score: 0.04175903648138046	 
Topic: 0.031*"vcenter" + 0.030*"path" + 0.030*"datastor" + 0.027*"server" + 0.025*"proghsz" + 0.023*"prci" + 0.023*"prog" + 0.023*"acvcsap" + 0.021*"servic" + 0.021*"user"


<b> Resolution Text

In [51]:
dict_Res = corpora.Dictionary(processed_Resolution)
count = 0
for k, v in dict_Res.iteritems(): # k -> key | v -> value
    print(k, v)
    count += 1
    if count > 10:
        break

0 abl
1 appear
2 confirm
3 contact
4 custom
5 dtype
6 environ
7 imt
8 issu
9 kyle
10 length


In [52]:
# dict_Res.filter_extremes(no_below=2, no_above=0.25)
Res_bow_corpus = [dict_Res.doc2bow(doc) for doc in processed_Resolution]

In [53]:
Res_tfidf = models.TfidfModel(Res_bow_corpus)
Res_corpus_tfidf = Res_tfidf[Res_bow_corpus]

for comment in Res_corpus_tfidf:
    pprint(comment)
    break

[]


In [54]:
Res_lda_model_tfidf = gensim.models.LdaMulticore(Res_bow_corpus, num_topics=4, id2word=dict_Res, passes=2, workers=4)
for idx, topic in Res_lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.130*"resolv" + 0.082*"custom" + 0.044*"ticket" + 0.043*"environ" + 0.042*"confirm" + 0.042*"object" + 0.042*"abl" + 0.042*"imt" + 0.042*"issu" + 0.042*"transfer"
Topic: 1 Word: 0.120*"resolv" + 0.083*"custom" + 0.044*"reop" + 0.043*"kyle" + 0.043*"success" + 0.043*"abl" + 0.043*"server" + 0.042*"length" + 0.042*"imt" + 0.042*"ticket"
Topic: 2 Word: 0.123*"resolv" + 0.083*"custom" + 0.043*"imt" + 0.043*"object" + 0.043*"note" + 0.042*"length" + 0.042*"issu" + 0.042*"dtype" + 0.042*"reop" + 0.042*"confirm"
Topic: 3 Word: 0.127*"resolv" + 0.085*"custom" + 0.043*"kyle" + 0.043*"multipleday" + 0.043*"normaliz" + 0.043*"issu" + 0.042*"contact" + 0.042*"transfer" + 0.042*"success" + 0.042*"dtype"


In [55]:
for index, score in sorted(Res_lda_model_tfidf[Res_corpus_tfidf[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, Res_lda_model_tfidf.print_topic(index, 10)))


Score: 0.25	 
Topic: 0.130*"resolv" + 0.082*"custom" + 0.044*"ticket" + 0.043*"environ" + 0.042*"confirm" + 0.042*"object" + 0.042*"abl" + 0.042*"imt" + 0.042*"issu" + 0.042*"transfer"

Score: 0.25	 
Topic: 0.120*"resolv" + 0.083*"custom" + 0.044*"reop" + 0.043*"kyle" + 0.043*"success" + 0.043*"abl" + 0.043*"server" + 0.042*"length" + 0.042*"imt" + 0.042*"ticket"

Score: 0.25	 
Topic: 0.123*"resolv" + 0.083*"custom" + 0.043*"imt" + 0.043*"object" + 0.043*"note" + 0.042*"length" + 0.042*"issu" + 0.042*"dtype" + 0.042*"reop" + 0.042*"confirm"

Score: 0.25	 
Topic: 0.127*"resolv" + 0.085*"custom" + 0.043*"kyle" + 0.043*"multipleday" + 0.043*"normaliz" + 0.043*"issu" + 0.042*"contact" + 0.042*"transfer" + 0.042*"success" + 0.042*"dtype"


## Visualizations

<b> Summary Text

In [56]:
pyLDAvis.enable_notebook()

In [57]:
Sum_TFIDFvis = pyLDAvis.gensim_models.prepare(Sum_lda_model_tfidf,Sum_bow_corpus, dict_Sum)

c:\miniconda37\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [58]:
Sum_TFIDFvis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.192907 -0.007319       1        1  30.590499
2      0.145369 -0.223477       2        1  30.017726
3      0.030907  0.297695       3        1  25.757218
0     -0.369182 -0.066899       4        1  13.634557, topic_info=          Term          Freq         Total Category  logprob  loglift
113   logwatch  36381.000000  36381.000000  Default  30.0000  30.0000
491  netbackup  19820.000000  19820.000000  Default  29.0000  29.0000
0         data  10662.000000  10662.000000  Default  28.0000  28.0000
1         hour  10597.000000  10597.000000  Default  27.0000  27.0000
2         past  10599.000000  10599.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
790  microsoft    901.269250   2520.122975   Topic4  -5.1358   0.9643
5        error   1054.100457  22089.636034   Topic4  -4.9792  -1.0499
16      servic   1005.107503  28732.906883   Topic4  -5.0268  -1.3604
180      alert    939.642102  12146.386412   Topic4  -5.0942  -0.5667
183    ciscouc    918.140188  11782.502857   Topic4  -5.1173  -0.5595

[244 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
609       1  0.991436   access
609       2  0.004077   access
609       3  0.001631   access
609       4  0.003261   access
63        1  0.000261  acvcsap
...     ...       ...      ...
774       4  0.003136   xrapoi
1200      1  0.001095   xrisch
1200      2  0.001095   xrisch
1200      3  0.997335   xrisch
1200      4  0.001095   xrisch

[772 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 4, 1])

<b> Resolution Text